# Bike share dataset
Liga con la descripción de los datos:
https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset

In [16]:
import requests
import pandas as pd
from zipfile import ZipFile


In [17]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip"
req = requests.get(url)
file_name = "Bike-Sharing-Dataset.zip"

with open(file_name, "wb") as my_file:
    my_file.write(req.content)
    
with ZipFile(file_name) as my_zip:
    my_zip.printdir()
    my_zip.extractall()

File Name                                             Modified             Size
Readme.txt                                     2013-12-20 13:29:02         5607
day.csv                                        2013-12-20 12:10:44        57569
hour.csv                                       2013-12-20 12:10:44      1156736


In [18]:
bikes_day = pd.read_csv("day.csv", index_col=0)
bikes_hour = pd.read_csv("hour.csv", index_col=0)

bikes_day


,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796
731,2012-12-31,1,1,12,0,1,1,2,0.215833,0.223487,0.577500,0.154846,439,2290,2729


In [19]:
bikes_hour

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61
17379,2012-12-31,1,1,12,23,0,1,1,1,0.26,0.2727,0.65,0.1343,12,37,49
